In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

In [18]:
df_titanic = pd.read_csv("./titanic.csv")
df_titanic

,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone
0,male,22.0,1,0,7.2500,Third,man,True,Southampton,no,False
1,female,38.0,1,0,71.2833,First,woman,False,Cherbourg,yes,False
2,female,26.0,0,0,7.9250,Third,woman,False,Southampton,yes,True
3,female,35.0,1,0,53.1000,First,woman,False,Southampton,yes,False
4,male,35.0,0,0,8.0500,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...
886,male,27.0,0,0,13.0000,Second,man,True,Southampton,no,True
887,female,19.0,0,0,30.0000,First,woman,False,Southampton,yes,True
888,female,32.0,1,2,23.4500,Third,woman,False,Southampton,no,False
889,male,26.0,0,0,30.0000,First,man,True,Cherbourg,yes,True


### Funcion: tipifica_variables

Esta función debe recibir como argumento un dataframe, un entero (`umbral_categoria`) y un float (`umbral_continua`). La función debe devolver un dataframe con dos columnas "nombre_variable", "tipo_sugerido" que tendrá tantas filas como columnas el dataframe. En cada fila irá el nombre de una de las columnas y una sugerencia del tipo de variable. Esta sugerencia se hará siguiendo las siguientes pautas:
+ Si la cardinalidad es 2, asignara "Binaria"
+ Si la cardinalidad es menor que `umbral_categoria` asignara "Categórica"
+ Si la cardinalidad es mayor o igual que `umbral_categoria`, entonces entra en juego el tercer argumento:
    * Si además el porcentaje de cardinalidad es superior o igual a `umbral_continua`, asigna "Numerica Continua"
    * En caso contrario, asigna "Numerica Discreta"

In [ ]:
def tipifica_variables(df, umbral_categoria, umbral_continua):
    """
    Basándonos en los umbrales de categoría y continua que le hemos pasado, la función
    decidirá sobre el tipo de variable que iremos a utilizar

    Pensada para cualquier tipo de DataFrame, con el objetivo de categorizar el tipo de
    cada variable que se encuentre en este

    Ponemos unas validaciones para evitar un posible error a la hora de introducir
    los datos en la función en cuestión

    Creamos la variable resultado y hacemos dropna() para que nos pueda servir también con
    DataFrames con nulos

    Calculamos la cardinalidad y en base a esta y a los umbrales, decidimos que tipo de
    variable será cada una de ellas

    Argumentos:
    df (pd.DataFrame):
    DataFrame que contiene los datos sobre los cuales se realizará el análisis.

    umbral_categoria:
    Pasamos un número, que debe de ser entero en este caso

    umbral_continua:
    Pasamos un float ahora.

    """
# Validar umbral_categoria
    if not isinstance(umbral_categoria, int) or umbral_categoria <= 0:
        print("umbral_categoria debe ser un número entero positivo")
        return None


# Validar umbral_continua
    if not isinstance(umbral_continua, float) or not (0 < umbral_continua <= 1):
        print("umbral_continua debe ser un número decimal entre 0 y 1")
        return None
# 2. DataFrame de salida

    resultado = pd.DataFrame(columns=["nombre_variable", "tipo_sugerido"])

# Número total de filas
    n_filas = len(df)

# 3. Bucle por columnas

    for col in df.columns:

# 3.1 Cardinalidad

        n_unicos = df[col].dropna().nunique()

# Num de filas validas (no Nan)
        n_filas_col = df[col].dropna().shape[0]

# 3.2 Proporción

        proporcion = n_unicos / n_filas if n_filas > 0 else 0

# 3.3 Decisión de tipo

# 3.3.1: Binaria
        if n_unicos == 2:
            tipo = "Binaria"


# 3.3.2: Categórica
        elif n_unicos < umbral_categoria:
            tipo = "Categórica"


# 3.3.3: Numéricas
        else:
# Continua
            if proporcion >= umbral_continua:
                tipo = "Numerica Continua"
# Discreta
            else:
                tipo = "Numerica Discreta"

# 3.4 Guardar resultado
        resultado.loc[len(resultado)] = [col, tipo]
# 4. Retorno
    return resultado

In [116]:
tipifica_variables(df_titanic, 5, 0.1)

,nombre_variable,tipo_sugerido
0,sex,Binaria
1,age,Numerica Discreta
2,sibsp,Numerica Discreta
3,parch,Numerica Discreta
4,fare,Numerica Continua
5,class,Categórica
6,who,Categórica
7,adult_male,Binaria
8,embark_town,Categórica
9,alive,Binaria


In [ ]:
def get_features_cat_regression(df, target_col, pvalue = 0.05):

    """
    Selecciona las columnas categóricas del DataFrame que presentan una relación
    estadísticamente significativa con nuestra variable objetivo numérica utilizando
    el test ANOVA.

    Pensada para problemas de Machine Learning de tipo regresión, donde
    la variable objetivo es numérica y las variables explicativas son
    categóricas.

    Argumentos:
    df (pd.DataFrame):
    DataFrame que contiene los datos sobre los cuales se realizará el análisis.

    target_col:
    Nombre de la columna del DataFrame que se utilizará como variable objetivo.
    Esta columna debe ser numérica (int o float), ya que representa el target
    de un modelo de regresión.

    pvalue:
    Umbral de significancia estadística para el test ANOVA. Por defecto es 0.05.
    Si el p-value obtenido en el test es menor que este valor, se considera que
    existe una relación estadísticamente significativa entre la variable
    categórica y el target.

    Retorna la lista con los nombres de las columnas categóricas que presentan una
    relación estadísticamente significativa con la variable objetivo. Si alguno
    de los parámetros de entrada no es válido, la función imprime un mensaje de
    error y retorna None.
    """

# Validación del DataFrame
    if not isinstance(df, pd.DataFrame):
        print("El primer argumento debe ser un DataFrame")
        return None
    
# Validación de la columna target
    if target_col not in df.columns:
        print("target_col no existe en el DataFrame")
        return None
    
# Validación de tipo numérico
    if df[target_col].dtype not in ['int64', 'float64']:
        print("target_col debe ser una variable numérica")
        return None
    
# Validación del p-value
    if not isinstance(pvalue, float) or not (0 < pvalue < 1):
        print("pvalue debe ser un float entre 0 y 1")
        return None
    
# Selección de columnas categóricas
    cat_cols = df.select_dtypes(include=['object', 'category']).columns

    selected_cols = []

    for col in cat_cols:
# Crear grupos para ANOVA
        groups = [
            df[df[col] == category][target_col].dropna()
            for category in df[col].dropna().unique()
        ]
# Validar que hay al menos dos grupos
        if len(groups) < 2:
            continue

# Ejecutar test ANOVA
        stat, p = f_oneway(*groups)

# Comparar con el umbral
        if p < pvalue:
            selected_cols.append(col)

    return selected_cols

In [99]:
get_features_cat_regression(df_titanic, "fare", 0.05)

['sex', 'class', 'who', 'embark_town', 'alive']